In [47]:
import pandas as pd
import numpy as np
import utils
import pyarrow

READ_PARQUET = True

In [48]:
import importlib
utils = importlib.reload(utils)  # reloads the utils module without restarting kernel

In [49]:
if READ_PARQUET:
    print("Reading parquet.")
    df = utils.load_parquet()
else:
    print("Reading csv and savings as parquet.")
    df = utils.load_csv()
    utils.save_parquet(df)

Reading parquet.


In [55]:
from copy import deepcopy
# drop uncomplete cases (2017 mostly) and thus do not have a complete label

#cases_df = deepcopy(df.loc[df['Complete Timestamp'].dt.year < 2017])
cases_df = deepcopy(df)

In [70]:
cases_df = cases_df.rename(columns = {'Complete Timestamp': 'Timestamp'})
cases_df.Timestamp = pd.to_datetime(cases_df.Timestamp)

## Undesired Outcome 1

In [ ]:
case_subset = self.data[self.data['Case ID'] == case_id]
            case_events = case_subset['Activity'].to_numpy()
            beginPaymentString = 'Payment application-Application-begin payment'
            abortPaymentString = 'Payment application-Application-abort payment'

            if beginPaymentString in case_events:
                # Receive the index of the last begin payment
                lastBeginPaymentIdx = np.where(case_events == beginPaymentString)[0].max()

                # Check whether the last begin payment is performed in the same year
                year_start = min(case_subset['Complete Timestamp']).year
                year_end = max(case_subset[case_subset['Activity'] == beginPaymentString]['Complete Timestamp']).year

In [76]:
start_year_case = cases_df.groupby(by=['Case ID'])['Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})
start_year_case['Case ID'] = start_year_case.index
start_year_case['StartYear'] = start_year_case['StartDate'].dt.year
start_year_case

,StartDate,Case ID,StartYear
Case ID,,,
0002505cb62792e4,2015-04-28,0002505cb62792e4,2015
0002a55a6130cec8,2015-04-17,0002a55a6130cec8,2015
0004ff62053a60ce,2015-05-11,0004ff62053a60ce,2015
000612b48d30de74,2015-05-06,000612b48d30de74,2015
0006cc909ce508b0,2016-04-28,0006cc909ce508b0,2016
...,...,...,...
fffa80507910b904,2017-05-10,fffa80507910b904,2017
fffa93db29ee0841,2016-05-12,fffa93db29ee0841,2016
fffab10d958a3e4d,2016-05-11,fffab10d958a3e4d,2016


In [ ]:
latest_begin_payment_year = [cases_df['Activity'] == 'Payment application-Application-abort payment'].groupby(by=['Case ID'])['Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})

# latest_begin_payment_year['_Latest_Begin_Payment_Year'] =  latest_begin_payment_year['StartDate'].dt.year
# latest_begin_payment_year = latest_begin_payment_year.reset_index()
# latest_begin_payment_year.head()

In [15]:
# determine case start year
start_year_case = cases_df.groupby(by=['Case ID'])['Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})
start_year_case['StartYear'] = start_year_case['StartDate'].dt.year
start_year_case = start_year_case.reset_index()
start_year_case.head()


,Case ID,StartDate,StartYear
0,0002505cb62792e4,2015-04-28,2015
1,0002a55a6130cec8,2015-04-17,2015
2,0004ff62053a60ce,2015-05-11,2015
3,000612b48d30de74,2015-05-06,2015
4,0006cc909ce508b0,2016-04-28,2016


In [16]:
# determine latest begin payment year
latest_begin_payment_year = cases_df.loc[cases_df['Activity'] == 'Payment application-Application-abort payment'].groupby(by=['Case ID'])['Timestamp'].agg(['min']).rename(columns={'min':'StartDate'})
latest_begin_payment_year['_Latest_Begin_Payment_Year'] =  latest_begin_payment_year['StartDate'].dt.year
latest_begin_payment_year = latest_begin_payment_year.reset_index()
latest_begin_payment_year.head()

,Case ID,StartDate,_Latest_Begin_Payment_Year
0,0002505cb62792e4,2015-12-17 14:58:57.997,2015
1,0002a55a6130cec8,2015-12-18 12:40:48.866,2015
2,0004ff62053a60ce,2015-12-16 16:43:14.024,2015
3,000612b48d30de74,2015-12-18 13:09:36.753,2015
4,000843b028f083e2,2015-12-17 14:57:53.610,2015


In [17]:
# merge two dataframes
year_df = pd.merge(latest_begin_payment_year, start_year_case, on='Case ID')
del year_df['StartDate_x'] # remove
del year_df[ 'StartDate_y'] # remove
# Case ID is the unique identifier in merged_df and thus no grouping by is necessary
year_df['LatePayment'] = np.where(year_df['_Latest_Begin_Payment_Year'] > year_df['StartYear'], 1 , 0)

In [18]:
# cases with Latest Begin Payment event in a later year than the case has started
year_df.loc[year_df['LatePayment'] == 1]

,Case ID,_Latest_Begin_Payment_Year,StartYear,LatePayment
898,095ffc8dd79aae27,2016,2015,1
1800,1247d4083d7d565d,2016,2015,1
1913,13404f41b918553d,2016,2015,1
1977,13bd492abca27e38,2015,2014,1
1985,13d5a04a7ccf8108,2016,2015,1
3828,260155273269fe3e,2016,2015,1
4710,2e0f33f7ae3fadd9,2015,2014,1
7828,4c2f30ce48103ded,2016,2015,1
8286,50b780b4500b950f,2016,2015,1
8303,50f57bfbcac17b65,2015,2014,1


#### Label cases with payment in a later year than in which case started as undesired outcome 1

In [19]:

late_payments = year_df['Case ID'].loc[year_df['LatePayment'] == 1].unique()

# loop over all Case ID's that have a late payment
for i in late_payments:
    row = cases_df.loc[cases_df['Case ID'] == i, :]
    cases_df.loc[cases_df['Case ID'] == i,'_Begin_Payment_Next_Year'] = 1

    cases_df.loc[cases_df['Case ID'] == i,'UndesiredOutcome1'] = 1


#### Create df: single out Case ID and UndesiredOutcome column

In [20]:
undesired_1 = cases_df[['Case ID', 'UndesiredOutcome1']]

### Save Undesired Outcomes as dataframe in parquet file

In [58]:
def _init_(self, data):
    self.data = cases_df

In [66]:
unique_id = cases_df['Case ID'].unique()

In [67]:
unique_id

array(['8b99873a6136cfa6', 'd038e6a5fcaed7a4', 'd0c4a4241daa1d89', ...,
       '75e5c760d8399329', '50d70b51ed3efcdc', 'ad5dfa0b929735be'],
      dtype=object)

In [63]:
def is_undesired_outcome1(x):
            case_id = x['Case ID']
            case_subset = self.data[self.data['Case ID'] == case_id]
            case_events = case_subset['Activity'].to_numpy()
            beginPaymentString = 'Payment application-Application-begin payment'
            abortPaymentString = 'Payment application-Application-abort payment'

            if beginPaymentString in case_events:
                # Receive the index of the last begin payment
                lastBeginPaymentIdx = np.where(case_events == beginPaymentString)[0].max()

                # Check whether the last begin payment is performed in the same year
                year_start = min(case_subset['Complete Timestamp']).year
                year_end = max(case_subset[case_subset['Activity'] == beginPaymentString]['Complete Timestamp']).year
                if year_start != year_end:
                    print("year start {}, year end {}".format(year_start, year_end))
                    return True
                # Check if the last begin payment was followed by an abort payment
                elif abortPaymentString in case_events[lastBeginPaymentIdx:]:
                    print("last begin payment was followed by an abort payment")
                    return True

                else:
                    return False

            else:
                print("No Begin Payment")
                return True

In [68]:
is_undesired_outcome1(unique_id)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices